In [48]:
import requests
import pandas as pd
import json
from dateutil import parser

In [3]:
API_KEY = "c6cbea0fb8436d3a63eec8700ac4fa76-5779ef2f8c1151739495c4ab99f2df69"
ACCOUNT_ID = "101-004-24733228-001" 
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [6]:
session = requests.Session()

In [8]:
session.headers.update({
    "Authorization" : f"Bearer {API_KEY}",
    "Content-Type" : "application/json"
})

In [20]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [21]:
response = session.get(url, params = None, data = None, headers = None)

In [22]:
response.status_code

200

In [23]:
data = response.json()

instruments_list = data['instruments']

In [24]:
instruments_list = data['instruments']

In [25]:
len(instruments_list)

124

In [27]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'minimumGuaranteedStopLossDistance', 'guaranteedStopLossOrderExecutionPremium', 'guaranteedStopLossOrderLevelRestriction', 'tags', 'financing'])

In [29]:
keys_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [30]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in keys_i}

In [32]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.03333333333333'}

In [33]:
with open("../data/instruments.json","w") as f:
    f.write(json.dumps(instruments_dict, indent = 2))

In [79]:
def fetch_candles(pair_name, count=10, granularity = 'H1'):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response =  session.get(url, params = params, data = None, headers = None)
    data = response.json()
    
    if response.status_code==200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
            
    return response.status_code, data

def get_candle_df(data):
    if len(data) == 0:
        return pd.DataFrame()

    final_data = []
    prices = ['mid','bid','ask']
    ohlc = ['o','l','h','c']
    for candle in data:
        if candle['complete']==False:
            continue
        temp = {}
        temp['time'] = parser.parse(candle['time'])
        temp['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                temp[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(temp)
    df = pd.DataFrame.from_dict(final_data) 
    return df

def create_data_file(pair_name, count = 10, granularity = 'H1'):
    code,data = fetch_candles(pair_name,count,granularity)
    if code!=200:
        print("Failed",pair_name,data)
        return
    if len(data)==0:
        print("No candles",pair_name)
    candles_df = get_candle_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [74]:
code, data = fetch_candles('EUR_USD',count = 20)
candles_df =get_candle_df(data)

In [80]:
create_data_file("EUR_USD", count = 10, granularity='H4')

EUR_USD H4 10 candles, 2023-02-02 06:00:00+00:00 2023-02-03 18:00:00+00:00


In [81]:
our_curr = ['EUR','USD','GBP','JPY','CHF','NZD','CAD','AUD']
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1","H4"]:
                create_data_file(pr, count = 4001,granularity = g)

EUR_USD H1 4001 candles, 2022-06-15 00:00:00+00:00 2023-02-03 21:00:00+00:00
EUR_USD H4 4001 candles, 2020-07-13 01:00:00+00:00 2023-02-03 18:00:00+00:00
EUR_GBP H1 4001 candles, 2022-06-15 01:00:00+00:00 2023-02-03 21:00:00+00:00
EUR_GBP H4 4001 candles, 2020-07-13 01:00:00+00:00 2023-02-03 18:00:00+00:00
EUR_JPY H1 4001 candles, 2022-06-15 01:00:00+00:00 2023-02-03 21:00:00+00:00
EUR_JPY H4 4001 candles, 2020-07-10 17:00:00+00:00 2023-02-03 18:00:00+00:00
EUR_CHF H1 4001 candles, 2022-06-15 01:00:00+00:00 2023-02-03 21:00:00+00:00
EUR_CHF H4 4001 candles, 2020-07-13 01:00:00+00:00 2023-02-03 18:00:00+00:00
EUR_NZD H1 4001 candles, 2022-06-15 00:00:00+00:00 2023-02-03 21:00:00+00:00
EUR_NZD H4 4001 candles, 2020-07-13 01:00:00+00:00 2023-02-03 18:00:00+00:00
EUR_CAD H1 4001 candles, 2022-06-15 01:00:00+00:00 2023-02-03 21:00:00+00:00
EUR_CAD H4 4001 candles, 2020-07-13 01:00:00+00:00 2023-02-03 18:00:00+00:00
EUR_AUD H1 4001 candles, 2022-06-15 01:00:00+00:00 2023-02-03 21:00:00+00:00

,time,volume,mid_o,mid_l,mid_h,mid_c,bid_o,bid_l,bid_h,bid_c,ask_o,ask_l,ask_h,ask_c
0,2023-02-03 02:00:00+00:00,2932,1.08938,1.08878,1.08961,1.08897,1.08931,1.08871,1.08953,1.08890,1.08946,1.08886,1.08970,1.08904
1,2023-02-03 03:00:00+00:00,1807,1.08896,1.08894,1.08959,1.08941,1.08889,1.08887,1.08952,1.08934,1.08904,1.08902,1.08966,1.08948
2,2023-02-03 04:00:00+00:00,1795,1.08941,1.08872,1.08948,1.08932,1.08933,1.08865,1.08942,1.08925,1.08949,1.08879,1.08955,1.08940
3,2023-02-03 05:00:00+00:00,2420,1.08930,1.08898,1.08984,1.08938,1.08922,1.08891,1.08977,1.08930,1.08938,1.08906,1.08992,1.08946
4,2023-02-03 06:00:00+00:00,2642,1.08938,1.08916,1.09001,1.08984,1.08931,1.08908,1.08993,1.08976,1.08945,1.08924,1.09009,1.08992
5,2023-02-03 07:00:00+00:00,5956,1.08982,1.08820,1.09062,1.08996,1.08975,1.08813,1.09055,1.08989,1.08990,1.08827,1.09068,1.09003
6,2023-02-03 08:00:00+00:00,6367,1.08998,1.08846,1.09075,1.09034,1.08992,1.08839,1.09067,1.09027,1.09005,1.08853,1.09083,1.09042
7,2023-02-03 09:00:00+00:00,5839,1.09036,1.08936,1.09312,1.09301,1.09029,1.08929,1.09304,1.09294,1.09043,1.08944,1.09319,1.09308
8,2023-02-03 10:00:00+00:00,4997,1.09299,1.09250,1.09368,1.09352,1.09292,1.09244,1.09360,1.09344,1.09306,1.09257,1.09375,1.09359
9,2023-02-03 11:00:00+00:00,4119,1.09353,1.09248,1.09372,1.09290,1.09345,1.09241,1.09365,1.09283,1.09361,1.09256,1.09379,1.09298
